---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/11_Americans.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
"""
import numpy as np

import numpy as np
from scipy.stats import norm
import scipy.optimize as optimize

def black_scholes_call(S, K, r, sigma, q, T):
"""
"""
    Inputs:
    S = initial stock price
    K = strike price
    r = risk-free rate
    sigma = volatility
    q = dividend yield
    T = time to maturity
"""
"""
    if sigma == 0:
        return max(0, np.exp(-q * T) * S - np.exp(-r * T) * K)
    else:
        d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        N1 = norm.cdf(d1)
        N2 = norm.cdf(d2)
        return np.exp(-q * T) * S * N1 - np.exp(-r * T) * K * N2


def american_call_dividend(S, K, r, sigma, Div, TDiv, TCall):
    LessDiv = S - np.exp(-r * TDiv) * Div
    if Div / K <= 1 - np.exp(-r * (TCall - TDiv)):
        return black_scholes_call(LessDiv, K, r, sigma, 0, TCall)

    upper = K
    while upper + Div - K < black_scholes_call(upper, K, r, sigma, 0, TCall - TDiv):
        upper *= 2

    tol = 1e-6
    lower = 0
    flower = Div - K
    fupper = upper + Div - K - black_scholes_call(upper, K, r, sigma, 0, TCall - TDiv)
    guess = 0.5 * lower + 0.5 * upper
    fguess = guess + Div - K - black_scholes_call(guess, K, r, sigma, 0, TCall - TDiv)
    while upper - lower > tol:
        if fupper * fguess < 0:
            lower = guess
            flower = fguess
            guess = 0.5 * lower + 0.5 * upper
            fguess = guess + Div - K - black_scholes_call(guess, K, r, sigma, 0, TCall - TDiv)
        else:
            upper = guess
            fupper = fguess
            guess = 0.5 * lower + 0.5 * upper
            fguess = guess + Div - K - black_scholes_call(guess, K, r, sigma, 0, TCall - TDiv)
    LessDivStar = guess

    d1 = (np.log(LessDiv / LessDivStar) + (r + sigma ** 2 / 2) * TDiv) / (sigma * np.sqrt(TDiv))
    d2 = d1 - sigma * np.sqrt(TDiv)
    d1prime = (np.log(LessDiv / K) + (r + sigma ** 2 / 2) * TCall) / (sigma * np.sqrt(TCall))
    d2prime = d1prime - sigma * np.sqrt(TCall)
    rho = -np.sqrt(TDiv / TCall)
    N1 = norm.cdf(d1)
    N2 = norm.cdf(d2)
    M1 = binormal_prob(-d1, d1prime, rho)
    M2 = binormal_prob(-d2, d2prime, rho)

    return LessDiv * N1 + np.exp(-r * TDiv) * (Div - K) * N2 + LessDiv * M1 - np.exp(-r * TCall) * K * M2

# Example usage

S = 100
K = 90
r = 0.05
sigma = 0.2
Div = 5
TDiv = 0.5
TCall = 1
N = 10

print("American Call with Dividend:", american_call_dividend(S, K, r, sigma, Div, TDiv, TCall))
"""